In [1]:
'''BARCLAYS OPTION TRADING STRATEGY'''
#This strat aims to exploit the gap between implied option vol and historical vol (volatility risk premium)
#Retail traders in the past years have created huge liquidity for short-dated calls on large cap stocks
#We build a strat around this that basically buys undervalued calls and sells overvalued ones
#The expiration date we select is going to be 2 weeks, as this is where most liquidiy is concentrated

'BARCLAYS OPTION TRADING STRATEGY'

In [2]:
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import dotenv
import os
import pandas as pd
import warnings
import datetime
import requests
from pymongo import MongoClient
from dateutil.relativedelta import relativedelta

warnings.filterwarnings('ignore')

dotenv.load_dotenv()
ALPHA_API = os.getenv("ALPHA_VANTAGE")
MONGO_URI = os.getenv("MONGO_URI")

client = MongoClient(MONGO_URI)
db = client['Barclays-Options']
collection = db['options-data']


In [3]:
'''Taking data from spy stocks to build the strat around'''
#We parse the wikipedia page for spy data

sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')
tickers = sp500['Symbol'].unique().tolist()

data = yf.download(tickers, period='1y')
data = data.stack()
data.index.names = ['date', 'ticker']
data.columns = data.columns.str.lower()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  503 of 503 completed


In [4]:
'''Well now filter by dollar volume to only opearte with the stocks which fit our liquidity constraints'''
#We will compute the mean dollar volume from the past year to filter liquid stocks
#Further, we selecct large cap stocks, as retail trader liquidity is more likely concentrated here

latest_date = data.index.get_level_values('date').max() #we use this below for option vol

data['dollarvol'] = (data['close'] * data['volume']) / 1e6
data = data.groupby('ticker').mean() #finding mean dollar vol over past year

data = data.sort_values(by='dollarvol', ascending = False)
data = data.head(20) #top 20 liquid stocks

In [5]:
'''Lets try to build a options volume column in the dataframe'''
#I tweaked with the weeks value and found the most liquidity to be in 2 weeks expriration date options, so we go with that
#This is an incredibly slow process, as yfinance doesn't give option volume directly and we have to find a workaround
#We now filter top 10 stocks with highest option volume
#Liquidity is either most concentrated in 1 week expiry options or 2 week expiry options

'''BUG => Too slow when trying to first filter by option volume, so did that later'''

data['option_volume'] = None
data['sector'] = None

latest_tickers = data.index.get_level_values('ticker')

count = 1

for tckr in latest_tickers:
    #expiration_date = (latest_date + timedelta(weeks=2)).strftime('%Y-%m-%d') #or tckr.options[1]

    tk = yf.Ticker(tckr)
    expiration_date = tk.options[1]
    opt_chain = tk.option_chain(expiration_date)

    option_volume = opt_chain.calls['volume'].sum() + opt_chain.puts['volume'].sum()

    data.loc[tckr, 'option_volume'] = option_volume
    data.loc[tckr, 'sector'] = tk.info.get('sector', 'NA')

    print("Ticker ", count, " done")
    count += 1

Ticker  1  done
Ticker  2  done
Ticker  3  done
Ticker  4  done
Ticker  5  done
Ticker  6  done
Ticker  7  done
Ticker  8  done
Ticker  9  done
Ticker  10  done
Ticker  11  done
Ticker  12  done
Ticker  13  done
Ticker  14  done
Ticker  15  done
Ticker  16  done
Ticker  17  done
Ticker  18  done
Ticker  19  done
Ticker  20  done


In [6]:
'''Checking how different max_dollar_volume and max_option_volume tickers are'''

data = data.sort_values(by='dollarvol', ascending = False)
max_dollar_vol = data.head(10)

data = data.sort_values(by='option_volume', ascending = False)
max_option_vol = data.head(10)

max_dvol= set(max_dollar_vol.index.to_list())
max_ovol = set(max_option_vol.index.to_list())

max_dvol.difference(max_ovol)
max_ovol.difference(max_dvol)

data = max_option_vol

In [7]:
'''Now, we will create a hashmap that maps each sector to its exchange traded fund(if it exists)'''
#We will need this later for the stat arb strategy
#There is no easy way to do this, so well select all popular etfs, view the sector of the top holding of each and create a map

etfs = [
    'XLK',  # Technology Select Sector SPDR Fund
    'XLF',  # Financial Select Sector SPDR Fund
    'XLV',  # Health Care Select Sector SPDR Fund
    'XLY',  # Consumer Discretionary Select Sector SPDR Fund
    'XLP',  # Consumer Staples Select Sector SPDR Fund
    'XLE',  # Energy Select Sector SPDR Fund
    'XLB',  # Materials Select Sector SPDR Fund
    'XLI',  # Industrial Select Sector SPDR Fund
    'XLU',  # Utilities Select Sector SPDR Fund
    'XLRE', # Real Estate Select Sector SPDR Fund
    'XLC',  # Communication Services Select Sector SPDR Fund
    'EEM',  # iShares MSCI Emerging Markets ETF
    'SPY',  # SPDR S&P 500 ETF Trust
    'VTI',  # Vanguard Total Stock Market ETF
    'VTV'   # Vanguard Value ETF
]

sector_to_etf = {}

for etf in etfs:
    etf_data = yf.Ticker(etf).funds_data.top_holdings
    etf_ticker = etf_data.index[0] #first holding
    sector = yf.Ticker(etf_ticker).info.get('sector', 'NA')
    sector_to_etf[sector] = etf

In [8]:
'''NOW COMES THE FUN QUANT PART'''
#We need a VolScore metric that can identify the spread of volatility risk premium
#Barclays doesn't reveal what they used to calculate volScore
#An educated guess (based on traditional pairs trading) is that we compare implied vol to historical vol AND sector vol
#Underlying assumption is that vol deviations are temporary (mean reversion)
#The formula I agree on is:

#            IV - (w1 * HV  +  w2 * SV)
# VolScore = --------------------------
#                    sigma_res

#w1 and w2 are weights that we'll determine with RollingOLS or Kalman Filter
#Weights should be time-varying preferably
#There is no need to weight IV too as the rationale is that IV already embeds the risk premium
#This comes from training GARCH models research - Univ of North Carolina research

# IV = w0 + w1 * HV + w2 * SV + epsilon_t
# epsilon_t is our error term, IV - IV_hat or residual
# IV is actual implied volatility and IV_hat is what our model estimates it to be
# The error term in the reason we normalize as we can't say if a given residue is significant or just market noise

#sigma_res is the standard deviation of residuals
#historical deviation of IV from weighted avg of HV and SV
#This normalization lets us use VolScore like a z-score, which lets us better gauge tradding opps

'''CONSIDERATIONS'''
#Use HFT data for HV as it helps capture intraday price movements that daily data can miss
#this reduces noise in data as shown by past research - Barndorff-Nielsen and Shephard
#Might have to do more research on how many years of data is best suited for pairs trading, curr is 1 yr

'CONSIDERATIONS'

In [9]:
'''Downloading etf data'''
#We will need this to compute the historical volatility of the sector
#We'll compute the daily volatilites for every day, mean them and find the annualized avg vol


req_etfs = [sector_to_etf[s] for s in data.sector.unique()]
tickers = " ".join(req_etfs)

etf_data = yf.download(tickers, period="3y")
etf_data = etf_data.stack()
etf_data.index.names = ['date', 'ticker']
etf_data.columns = etf_data.columns.str.lower()

[*********************100%***********************]  3 of 3 completed


In [10]:
'''Computing historical volatility'''

def calculate_hv(df, col):
    df = df.sort_index(level='date')
    
    returns = df['close'] / df['close'].shift(1)
    df['log_returns'] = np.log(returns)
    rolling_vol = df['log_returns'].rolling(window=252).std()
    df[col] = rolling_vol * np.sqrt(252)  # annual vol
    
    return df

In [11]:
# etff = yf.download('XLY', period='1mo')
# etff = etff.sort_values(by = "Date", ascending=False)
# etff = etff.head(2)
# print(etff.Close.XLY[-1], etff.Close.XLY[0])
# np.log(etff.Close.XLY[0] / etff.Close.XLY[-1])

In [12]:
'''Now we download the data for the 10 stocks we selected'''
#These contain all the actual option contracts we will be trading
#The dataframe consists of a IV section, which we'll use in the model we created

tickers = " ".join(data.index.tolist())

options_data = yf.download(tickers, period="3y")
options_data = options_data.stack()
options_data.index.names = ['date', 'ticker']
options_data.columns = options_data.columns.str.lower()

[*********************100%***********************]  10 of 10 completed


In [13]:
etf_data = etf_data.groupby("ticker").apply(calculate_hv, 'HV').dropna()
options_data = options_data.groupby("ticker").apply(calculate_hv, 'RV').dropna()

In [14]:
'''Run this script only once, otherwise necessary cols will be deleted'''

etf_data = etf_data.reset_index(level=2, drop=True) #removing duplicate ticker index
options_data = options_data.reset_index(level=2, drop=True)

In [15]:
'''Modifying the options_data dataframe to include the HV for the corresponding ticker-etf pair'''
#BUG: for loop was very slow here, so i ended up using previous dataframes mapping

options_data['sector'] = options_data.index.get_level_values('ticker').map(data['sector'])
options_data['ETF'] = options_data['sector'].map(sector_to_etf)

func = lambda row: etf_data.loc[(row['ETF'], row.name[1]), 'HV'] if (row['ETF'], row.name[1]) in etf_data.index else None

options_data['SV'] = options_data.apply(func, axis=1)

options_data.drop(columns=['ETF', 'sector'], axis=1, inplace=True)
options_data = options_data.dropna()

In [29]:
'''Downloading options data for past two years to compute VIX'''
#Im using alphavantage API as this was the best service that I could find for free
#However alphavantage doesn't have a good way to directly fetch data as a dataframe
#It gives responses in json, which Im manually converting into a pd Df
#Because alphavantage rate limits (25 per day), ill store data to a MongoDB cluster first

two_yrs = (datetime.datetime.today() - relativedelta(years=2)).strftime('%Y-%m-%d')

collection.delete_many({}) #deleting all the 10 stocks for now

for ticker in data.index.get_level_values('ticker').tolist():
    url = 'https://www.alphavantage.co/query?' \
    'function=HISTORICAL_OPTIONS&symbol='+ticker+'&apikey='+ALPHA_API+'&date='+two_yrs

    r = requests.get(url)

    push_data = r.json()
    push_data['_id'] = ticker

    collection.insert_one(push_data)

    print("Data pushed to cluster for " + ticker)

Data pushed to cluster for NVDA
Data pushed to cluster for INTC
Data pushed to cluster for TSLA
Data pushed to cluster for AAPL
Data pushed to cluster for PLTR
Data pushed to cluster for AMZN
Data pushed to cluster for META
Data pushed to cluster for AMD
Data pushed to cluster for SMCI
Data pushed to cluster for MSFT


In [16]:
'''Script to convert json data to Pandas dataframe'''
'''Script to query for data of a specific ticker from our Mongo cluster'''

def json_to_df(json_data): #json_data should be r.json()['data']
    ticker_options = pd.DataFrame()
    for day_data in json_data:
        ticker_options = pd.concat([ticker_options, pd.DataFrame([day_data])], ignore_index=True)

    return ticker_options

def get_data(ticker):
    mongo_data = collection.find_one({"_id": ticker})
    if mongo_data is None:
        return "Data not found for " + ticker
    else:
        return json_to_df(mongo_data['data']).sort_values(by='expiration')

In [17]:
'''Now we need to compute the implied volatitlies of each ticker'''
#We run into a problem where each option has option contracts with differing volatilities
#A simple average wont cut it as options differ in ATM and ITM status, and in moneyness
#A weighted average is a good place to start
#We can also build our own VIX like model

# The formula is: 
#    sigma² = (2 * e^(r*T) / T) * Σ [ (ΔK_i / K_i²) * Q(K_i) ] - (1/T) * ((F/K₀ - 1)²)

# T is the time to expiration in years (typically 30/365 for 30 days), and r is the risk-free rate.
# ΔK_i represents the interval between adjacent strikes, K_i are the strike prices, and Q(K_i) are the option mid-prices.
# F is the forward price of the underlying asset, and K₀ is the first strike below F.
# Taking the square root of sigma² gives the implied volatility (IV), in the same annualized units as HV and RV.

'Now we need to compute the implied volatitlies of each ticker'

In [18]:
'''Computing IV for each row in options data'''
#Well use the above formula with a 30 day rolling window and the option data in our DB to do this

#BUG => The data quality seems to be very poor as there are unrealistic striker for TSLA ($1.67)
#So, ill clean the data by going through each needed column and dropping outliers
#however i also need to ensure that my model works for tail events, and properly need to only remove bad data, while still keeping the tail event data

#perhaps its just best to use weighted average with open interest as weights, the free data seems too poor quality

def clean_data(df, cols, lower=0.01, upper=0.99):
    for c in ['implied_volatility', 'volume', 'open_interest']:
        df[c] = pd.to_numeric(df[c])

    for col in cols:
        # Convert col vals to numeric coercing errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # drop rows where the conversion failed (NaN values)
        df = df[df[col].notna()]
        
        l = df[col].quantile(lower)
        u = df[col].quantile(upper)
        df = df[(df[col] >= l) & (df[col] <= u)]
    return df

In [19]:
'''Computing weighted averages for each ticker and appending that to our options_data'''
#The weighted average formula we use weights both the open interest and volume of traded options

for t in options_data.index.get_level_values('ticker').unique().tolist():
    raw_data = get_data(t)
    cleaned_data = clean_data(raw_data, ['strike', 'bid', 'ask'])
    weighted_avg = lambda df: pd.Series(
            (df['implied_volatility'] * (df['volume'] + df['open_interest'])).sum() / 
            (df['volume'] + df['open_interest']).sum(), 
            index=df.index)

    cleaned_data['IV'] = cleaned_data.groupby('expiration').apply(weighted_avg).reset_index(level=0, drop=True)

    print("Data fetched and cleaned for", t)

    ticker_df = options_data.loc[t].reset_index()
    if 'IV' in ticker_df.columns:
        ticker_df.drop('IV', axis=1, inplace=True)

    ticker_df['date'] = pd.to_datetime(ticker_df['date'])
    cleaned_data['expiration'] = pd.to_datetime(cleaned_data['expiration'])

    ticker_df = ticker_df.sort_values('date')
    cleaned_data = cleaned_data.sort_values('expiration')

    ticker_df['IV'] = pd.merge_asof(ticker_df, 
                                    cleaned_data[['expiration', 'IV']], 
                                    left_on='date', 
                                    right_on='expiration', 
                                    direction='nearest')['IV']

    ticker_df = ticker_df.set_index(['date'])
    options_data.loc[t, 'IV'] = ticker_df['IV'].values

    print("Implied volatliity calculated for", t, "\n")

Data fetched and cleaned for AAPL
Implied volatliity calculated for AAPL 

Data fetched and cleaned for AMD
Implied volatliity calculated for AMD 

Data fetched and cleaned for AMZN
Implied volatliity calculated for AMZN 

Data fetched and cleaned for INTC
Implied volatliity calculated for INTC 

Data fetched and cleaned for META
Implied volatliity calculated for META 

Data fetched and cleaned for MSFT
Implied volatliity calculated for MSFT 

Data fetched and cleaned for NVDA
Implied volatliity calculated for NVDA 

Data fetched and cleaned for PLTR
Implied volatliity calculated for PLTR 

Data fetched and cleaned for SMCI
Implied volatliity calculated for SMCI 

Data fetched and cleaned for TSLA
Implied volatliity calculated for TSLA 



In [20]:
'''Building the linear regression model'''
#Now we build the linear regression model for VolScore based on the formula above
#we use HV from etf_data and RV from options_data for the weights

from statsmodels.regression.rolling import RollingOLS
import statsmodels.api as sm

def compute_volscore_ticker(df, wndw=252):
    df = df.sort_index(level = 'date').copy()

    X = sm.add_constant(df[['RV', 'SV']])
    Y = df['IV']
    rolling_model = RollingOLS(Y, X, window=wndw)
    results = rolling_model.fit().params

    #IV_hat is constant + coeffs * values
    df['w1'] = results.iloc[:, 1]
    df['w2'] = results.iloc[:, 2]
    df['b'] = results.iloc[:, 0]

    df['IV_hat'] = df['b'] + df['w1'] * df['RV'] + df['w2'] * df['SV']
    
    df['residual'] = Y - df['IV_hat']
    df['sigma_res'] = df['residual'].rolling(window=wndw, min_periods=1).std()

    df['VolScore'] = (df['IV'] - df['IV_hat']) / df['sigma_res']

    return df

def compute_volscore(df, wndw=252):
    volscore_fn = lambda x: compute_volscore_ticker(x, wndw=wndw)
    return df.copy().groupby(level = 'ticker').apply(volscore_fn)

In [21]:
volscore_data = compute_volscore(options_data, wndw=252).dropna()
volscore_data = volscore_data.reset_index(level=1, drop=True)

In [22]:
'''Building the signal generator functions'''
#Now for each of the 10 stocks, we fetch their option_chain data using yFinance and run our weights and biases to generate volscore based signals
#Before that though, we build the tradable_options dataframe

def option_chain_df(tck):
    t = yf.Ticker(tck)

    week1_calls, week2_calls = t.option_chain(t.options[0]).calls, t.option_chain(t.options[1]).calls
    week1_calls['expiration'], week2_calls['expiration'] = t.options[0], t.options[1]

    #week1_puts, week2_puts = t.option_chain(t.options[0]).puts, t.option_chain(t.options[1]).puts
    #week1_puts['expiration'], week2_puts['expiration'] = t.options[0], t.options[1]
    
    calls_chain = pd.concat([week1_calls, week2_calls], ignore_index=True)
    calls_chain['type'] = 'call'

    #puts_chain = pd.concat([week1_puts, week2_puts], ignore_index=True)
    #puts_chain['type'] = 'put'

    #options_chain = pd.concat([calls_chain, puts_chain], ignore_index=True)
    options_chain = pd.concat([calls_chain], ignore_index=True)
    options_chain['ticker'] = tck

    return options_chain

def get_ticker_params(tck):
    ticker_df = volscore_data.loc[tck].reset_index().sort_values('date', ascending=False)
    #ticker_df = ticker_df[ticker_df['date'] >= (datetime.datetime.today() - datetime.timedelta(days=1))]
    ticker_df = ticker_df.head(1) #Just the first value

    IV_hat = float(ticker_df['IV_hat'])
    sigma_res = float(ticker_df['sigma_res'])

    print(IV_hat, sigma_res)
    return IV_hat, sigma_res, ticker_df

In [23]:
'''Building the tradable options dataframe'''
#We only trade the top 20 liquid options for each ticker

tradable_options = pd.DataFrame()
tickers = volscore_data.index.get_level_values('ticker').unique().tolist()

for tck in tickers:
    options_chain = option_chain_df(tck)
    IV_hat, sigma_res, ticker_df = get_ticker_params(tck)

    options_chain['liquidity'] = options_chain['openInterest'] + options_chain['volume']
    options_chain = options_chain.sort_values('liquidity', ascending=False)
    options_chain = options_chain.head(20)

    Y = options_chain['impliedVolatility']

    options_chain['VolScore'] = (Y - IV_hat) / sigma_res

    tradable_options = pd.concat([tradable_options, options_chain.set_index('ticker', append=True)])

    print("Data appended for", tck)

0.3023925009802923 0.01047342997578033
Data appended for AAPL
0.5002430992402703 0.004177926897932742
Data appended for AMD
0.37411453650949694 0.0058971920286621385
Data appended for AMZN
0.358646667128917 0.006055123361037187
Data appended for INTC
0.45174571519093565 0.011005344464573597
Data appended for META
0.304171516001944 0.0024498845520732806
Data appended for MSFT
0.5398090019304517 0.010907979042744211
Data appended for NVDA
0.5892498192701279 0.01185055470458046
Data appended for PLTR
0.6463234947490194 0.0021351491933538106
Data appended for SMCI
0.6115975953415044 0.006867794819970931
Data appended for TSLA


In [27]:
long_options = tradable_options.sort_values(by='VolScore').head(10)
short_options = tradable_options.sort_values(by='VolScore', ascending=False).head(10)

portfolio_df = pd.concat([long_options, short_options])
portfolio_df

,,contractSymbol,lastTradeDate,strike,lastPrice,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,contractSize,currency,expiration,type,liquidity,VolScore
,ticker,,,,,,,,,,,,,,,,,,
120,MSFT,MSFT250328C00420000,2025-03-14 19:56:05+00:00,420.0,0.29,0.27,0.30,0.04,15.999996,3190.0,3716.0,0.228523,False,REGULAR,USD,2025-03-28,call,6906.0,-30.878262
33,MSFT,MSFT250321C00405000,2025-03-14 19:59:45+00:00,405.0,0.59,0.57,0.61,0.15,34.090904,5533.0,4186.0,0.233772,False,REGULAR,USD,2025-03-21,call,9719.0,-28.735724
35,MSFT,MSFT250321C00410000,2025-03-14 19:59:59+00:00,410.0,0.26,0.23,0.27,0.03,13.043472,1826.0,8418.0,0.235359,False,REGULAR,USD,2025-03-21,call,10244.0,-28.087979
31,MSFT,MSFT250321C00400000,2025-03-14 19:59:49+00:00,400.0,1.32,1.28,1.35,0.53,67.088610,11137.0,11505.0,0.237801,False,REGULAR,USD,2025-03-21,call,22642.0,-27.091450
37,MSFT,MSFT250321C00415000,2025-03-14 19:59:27+00:00,415.0,0.11,0.10,0.12,-0.03,-21.428572,2570.0,7088.0,0.240242,False,REGULAR,USD,2025-03-21,call,9658.0,-26.094921
29,MSFT,MSFT250321C00395000,2025-03-14 19:59:39+00:00,395.0,2.72,2.59,2.85,1.24,83.783780,6064.0,4416.0,0.253059,False,REGULAR,USD,2025-03-21,call,10480.0,-20.863142
28,MSFT,MSFT250321C00390000,2025-03-14 19:59:59+00:00,390.0,4.70,4.65,4.85,2.12,82.170540,7245.0,3626.0,0.257576,False,REGULAR,USD,2025-03-21,call,10871.0,-19.019562
23,AMD,AMD250321C00105000,2025-03-14 19:59:58+00:00,105.0,0.88,0.88,0.90,0.22,33.333330,11814.0,15494.0,0.420904,False,REGULAR,USD,2025-03-21,call,27308.0,-18.990009
26,AMD,AMD250321C00108000,2025-03-14 19:59:02+00:00,108.0,0.37,0.37,0.38,0.04,12.121209,10156.0,2701.0,0.422857,False,REGULAR,USD,2025-03-21,call,12857.0,-18.522527
